# Advanced Coding

## Introduction

This chapter covers some more advanced programming concepts. It's not strictly necessary to master the content of this chapter for subsequent chapters but it's here in case you want a deeper understanding or in case you find that you eventually need to draw on more sophisticated programming tools and concepts.

```{note}
You can safely skip this chapter and come back to it later.
```

This chapter has benefitted from the online book [*Research Software Engineering with Python*](https://merely-useful.github.io/py-rse/).

## Truthy and falsy values

Python objects can be used in expressions that will return a boolean value, such as when a list, `listy`, is used with `if listy`. Built-in Python objects that are empty are usually evaluated as `False`, and are said to be 'Falsy'. In contrast, when these built-in objects are not empty, they evaluate as `True` and are said to be 'truthy'.

(If you are building your own classes, you can define this behaviour for them through the `__bool__` dunder method.)

Let's see some examples:

In [2]:
def bool_check_var(input_variable):
    if not (input_variable):
        print('Falsy')
    else:
        print('Truthy')

listy = []
other_listy = [1, 2, 3]


bool_check_var(listy)

Falsy


In [3]:
bool_check_var(other_listy)

Truthy


The method we defined doesn't just operate on lists; it'll work for many various other truthy and falsy objects:

In [4]:
bool_check_var(0)

Falsy


In [5]:
bool_check_var([0, 0, 0])

Truthy


Note that zero was falsy, its the nothing of a float, but a list of three zeros is not an empty list, so it evaluates as truthy.

In [6]:
bool_check_var({})

Falsy


In [7]:
bool_check_var(None)

Falsy


Knowing what is truthy or falsy is useful in practice; imagine you'd like to default to a specific behaviour if a list called `list_vals` doesn't have any values in. You now know you can do it simply with `if list_vals`.

## Errors and exceptions

When a programme goes wrong, it throws up an error and halts. You won't be coding for long before you hit one of these errors, which have special names depending on what triggered them.

Let's see a real-life error in action:

In [8]:
denom = 0

print(1/denom)

ZeroDivisionError: division by zero

Oh no! We got a `ZeroDivisionError` and our programme crashed. Note that the error includes a 'Traceback' to show which line went wrong, which is helpful for debugging.

In practice, there are often times when we know that an error *could* arise, and we would like to specify what should happen when it does (rather than having the programme crash). 

We can use *exceptions* to do this. These come in a `try` ... `except` pattern, which looks like an `if` ... `else` pattern but applies to errors. If no errors occur inside the `try` block, the `except` block isn’t run but *if* something goes wrong inside the `try` then the `except` block is executed. Let's see an example:

In [9]:
for denom in [-5, 0, 5]:
    try:
        result = 1/denom
        print(f'1/{denom} == {result}')
    except:
        print(f'Cannot divide by {denom}')

1/-5 == -0.2
Cannot divide by 0
1/5 == 0.2


Now we can see two differences. First: the code executed just fine *without* halting. Second: when we hit the error, the `except` block was executed and told us what was going on.

In this case, we wrote an informative message about the error but it's convenient to use Python's built in messages where we can. In the below, not only do we send our own message about the error but we add info on what caused the error for the language too:

In [11]:
for denom in [-5, 0, 5]:
    try:
        result = 1/denom
        print(f'1/{denom} == {result}')
    except Exception as error:
        print(f'{denom} has no reciprocal; error is: {error}')

1/-5 == -0.2
0 has no reciprocal. Error is: division by zero
1/5 == 0.2


Sadly, division by zero is just one of the many errors you might encounter. What if a function is likely to end up running into several different errors? We can have multiple `except` clauses to catch these:

In [13]:
numbers = [-5, 0, 5]
for i in [0, 1, 2, 3]:
    try:
        denom = numbers[i]
        result = 1/denom
        print(f'1/{denom} == {result}')
    except IndexError as error:
        print(f'index {i} out of range; error is {error}')
    except ZeroDivisionError as error:
        print(f'{denom} has no reciprocal; error is: {error}')

1/-5 == -0.2
0 has no reciprocal; error is: division by zero
1/5 == 0.2
index 3 out of range; error is list index out of range


A full list of built-in errors may be [found here](https://docs.python.org/3/library/exceptions.html#exception-hierarchy) and they are nested in classes (eg `ZeroDivisionError` is a special case of a `ArithmeticError`).

Where do these errors come from anyway? What tells the programming language to throw a tantrum when it encounters certain combinations of values and operations.

The answer is that the person or people who wrote the code that's 'under the hood' can specify when such errors should be raised. Remember, the philosophy of Python is that things should faily loudly (so that they do not cause issues downstream). Here's an example of some code that raises its own errors using the `raise` keyword:

In [14]:
for number in [1, 0, -1]:
    try:
        if number < 0:
            raise ValueError(f'no negatives: {number}')
        print(number)
    except ValueError as error:
        print(f'exception: {error}')

1
0
exception: no negatives: -1


A `ValueError` is a built-in type of error and there are plenty of ones to choose from for your case. Some big or specialised libraries define their own types of error too.

One very clever feature of Python's exception handling is "throw low, catch high", which means that even if an error gets thrown way deep down in the middle of a code block, the catching exception can be used some way away.

Here's an example: the error arises *within* the `sum_reciprocals` function, but is caught elsewhere.

In [15]:
def sum_reciprocals(values):
    result = 0
    for v in values:
        result += 1/v
    return result

numbers = [-1, 0, 1]
try:
    one_over = sum_reciprocals(numbers)
except ArithmeticError as error:
    print(f'Error trying to sum reciprocals: {error}')

Error trying to sum reciprocals: division by zero
